<a href="https://colab.research.google.com/github/dgoldblum/Peptide_Charge_Prediction/blob/main/Final_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For my final project I will be using compiled sequence data from Xu et al. The data is composed of charge state percentages of peptides from extracted mass spec data. This model uses the sequence data to predict charge state percantages

In [ ]:
!pip install pyteomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Numpy, SciKitLearn, Pandas, and Pyteomics are the libraries used for this model.

In [ ]:
from pandas.core.arrays.sparse.array import Sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import pyteomics.mass as pmass
from pyteomics import parser

import io


uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['csd.csv'])) #make sure this is named correctly or it wont work
df.columns = ['Sequence', '1', '2', '3', '4', '5']
df

Saving csd.csv to csd (2).csv


,Sequence,1,2,3,4,5
0,AAAAAAALQAK,0.0,1.00000,0.00000,0.0,0.0
1,AAAAAWEEPSSGNGTAR,0.0,0.88317,0.11683,0.0,0.0
2,AAAAQFQVPWLESVLIVVSNNIDEEALAR,0.0,0.00000,1.00000,0.0,0.0
3,AAAASAAEAGIATTGTEDSDDALLK,0.0,0.00000,1.00000,0.0,0.0
4,AAAASVPNADGLK,0.0,1.00000,0.00000,0.0,0.0
...,...,...,...,...,...,...
21421,YQQQYLQPLPLTHYELVTDAFGHR,0.0,0.00000,0.00000,1.0,0.0
21422,YRGSIHDFPGFDPNQDAEALYTAMK,0.0,0.00000,0.00000,1.0,0.0
21423,YSIAFPLLcamCTHFMScamCTHELcamCPEER,0.0,0.00000,0.00000,1.0,0.0
21424,YTDPHNPHSPFNTAVADYWMPVDLYIGGK,0.0,0.00000,0.00000,1.0,0.0


Building the dataset involved creating a library of all possible amino acids (using one letter codes) and updating the mass compsition library with the different possible modification compostions. Methods could then be written to calculate peptide mass, length, modification, and basic sites. These methods create the matrix features for regression.

In [ ]:
#Vocabularly and Composition Update were taken from Xu et al. open source code.
aa_vocabulary = 'QNAVILFMYWPGCSTRKHED'

aa_comp = dict(pmass.std_aa_comp)
aa_comp.update({
    'ac'  : pmass.Composition({ 'H' : 3, 'C' : 2, 'O' : 1 }),
    'ox'   : pmass.Composition({ 'O' : 1 }),
    'cam'  : pmass.Composition({ 'H' : 3, 'C' : 2, 'N' : 1, 'O' : 1 }),
    'nem'  : pmass.Composition({ 'H' : 7, 'C' : 6, 'N' : 1, 'O' : 2 }),
    })


#Feature building methods

def pepMass(sequence):
  arr = []
  for i in sequence:
    contains = parser.parse(i, split = True, labels = aa_comp)
    seq = pmass.calculate_mass(contains, aa_comp = aa_comp)
    arr.append(seq)
  return arr

def pepLength(sequence):
  arr = []
  for i in sequence:
    arr.append(parser.length(i))
  return arr

#Basic Site count were taken from Xu et al. open source code.
def count_basic_sites(sequence):
  arr = []
  for i in sequence:
    counts = 1 + i.count('R') + i.count('H') + i.count('K') - i.count('ac')
    arr.append(counts)
  return arr


def isMod(sequence):
  arr = []
  for i in sequence:
    if i.isupper():
      arr.append(0)
    else:
      arr.append(1)
  return arr

def aaCount(sequence):
  aa = parser.parse(aa_vocabulary)
  temp = []
  for i in sequence:
    contains = parser.amino_acid_composition(i)
    temp.append(pd.DataFrame(data = contains, columns=aa, index = [i]))
  aa_count = pd.concat(temp, ignore_index=True)
  aa_count = aa_count.fillna(0)
  return aa_count





In [ ]:
#Getting all my data from the sequence
y = df.drop('Sequence', axis=1)
sequence_col = df['Sequence'].str.replace('ac-', 'ac')



#Calling all the methods to get our features
aa_comp_mat = aaCount(sequence_col)
mod_col = isMod(sequence_col)
basic_sites = count_basic_sites(sequence_col)
length_col = pepLength(sequence_col)
mass_col = pepMass(sequence_col)

Here all the features are compined into a large pandas dataframe. The frame consists of an intercept and 24 features for over 21,000 samples.

In [ ]:
#Creating our X matrix
X = pd.DataFrame(columns = ['Intercept', 'Mass', 'Length', 'B_Sites', 'Modified'])
X['Intercept'] = np.ones(df.shape[0])
X['Mass'] = mass_col
X['Length'] = length_col
X['B_Sites'] = basic_sites
X['Modified'] = mod_col

X = pd.concat([X, aa_comp_mat], axis = 1)

#plt.hist(y)

#Splitting the data
xTrain, xTest, yTrain_Pre, yTest_Pre = train_test_split(X, y, test_size = .3) #change the test size


#Classifying the y data as the largest charge state because the regression requries a 1d array. Did thi after train test split for restults comparisons later.
yTrain = np.argmax(np.asarray(yTrain_Pre), axis = 1)+1
yTest = np.argmax(np.asarray(yTest_Pre), axis = 1)+1

In order to predict charge state we need to do a multinomial regression - its essentially the same as our logistic regression but with multiple outcomes rather than our usual 0 or 1 outputs. The outputs are charge states 1-5. Here we use SciKitLearn to create our models.

In [ ]:
#Cross Entropy Loss against One v Rest
model_ncg = LogisticRegression(multi_class='multinomial', solver = 'newton-cg', max_iter=15000).fit(xTrain, yTrain)
model_ncg_ovr = LogisticRegression(multi_class='ovr', solver = 'newton-cg', max_iter=15000).fit(xTrain, yTrain)

#Creating our cross validation parameters
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=1)

#Scoring each of our models
ncg_score = cross_val_score(model_ncg, xTest, yTest, scoring='accuracy', cv=cv, n_jobs=-1)
ncg_score_ovr = cross_val_score(model_ncg_ovr, xTest, yTest, scoring='accuracy', cv=cv, n_jobs=-1)

#Checking mean model score over n_splits for each model
print(f'Mean Accuracy for newton-cg model is {np.mean(ncg_score)} and Standard Deviation is {np.std(ncg_score)}')
print(f'Mean Accuracy for newton-cg model is {np.mean(ncg_score_ovr)} and Standard Deviation is {np.std(ncg_score_ovr)}')

Mean Accuracy for newton-cg model is 0.9472101223812485 and Standard Deviation is 0.0040633764829205036
Mean Accuracy for newton-cg model is 0.886330636797345 and Standard Deviation is 0.005044756899819677


With the Cross Entropy Loss having a higher accuracy, we can now test how it works with different optimizers.

In [ ]:
#Checking 2 other optimizers
model_lbfgs = LogisticRegression(multi_class='multinomial', solver = 'lbfgs', max_iter=15000).fit(xTrain, yTrain)
model_saga = LogisticRegression(multi_class='multinomial', solver = 'saga', max_iter=15000).fit(xTrain, yTrain)

lbfgs_score = cross_val_score(model_lbfgs, xTest, yTest, scoring='accuracy', cv=cv, n_jobs=-1)
saga_score = cross_val_score(model_saga, xTest, yTest, scoring='accuracy', cv=cv, n_jobs=-1)

print(f'Mean Accuracy for lbfgs model is {np.mean(lbfgs_score)} and Standard Deviation is {np.std(lbfgs_score)}')
print(f'Mean Accuracy for saga model is {np.mean(saga_score)} and Standard Deviation is {np.std(saga_score)}')

Mean Accuracy for lbfgs model is 0.9462767060775773 and Standard Deviation is 0.003987218997457181
Mean Accuracy for saga model is 0.7287907073221324 and Standard Deviation is 0.0059352432805743665


In [ ]:
#Getting predicted values from our xTest to manually check our error
yHatC = model_ncg.predict(xTest)
errorC = (yTest-yHatC)**2
print(np.mean(errorC))

#Checking the error of a less accurate model
yHatS = model_saga.predict(xTest)
errorS = (yTest-yHatS)**2
print(np.mean(errorS))

#Looking at probabilities matrix, should look like our original y
yProb = model_ncg.predict_proba(X)
#yHat = pd.DataFrame(yProb, columns = ['1', '2', '3', '4', '5'])


np.round(yProb, decimals = 3)
print(y)

0.0499377722464219
0.23133167392657125
[[0.999 0.001 0.    0.   ]
 [0.933 0.067 0.    0.   ]
 [0.    1.    0.    0.   ]
 ...
 [0.    0.008 0.984 0.008]
 [0.    0.004 0.986 0.01 ]
 [0.    0.001 0.984 0.014]]
         1        2        3    4    5
0      0.0  1.00000  0.00000  0.0  0.0
1      0.0  0.88317  0.11683  0.0  0.0
2      0.0  0.00000  1.00000  0.0  0.0
3      0.0  0.00000  1.00000  0.0  0.0
4      0.0  1.00000  0.00000  0.0  0.0
...    ...      ...      ...  ...  ...
21421  0.0  0.00000  0.00000  1.0  0.0
21422  0.0  0.00000  0.00000  1.0  0.0
21423  0.0  0.00000  0.00000  1.0  0.0
21424  0.0  0.00000  0.00000  1.0  0.0
21425  0.0  0.00000  0.00000  1.0  0.0

[21426 rows x 5 columns]


In [ ]:
#Messing with things to put on the presentation
yHat = pd.DataFrame(columns = ['1'])
yHat['1'] = np.zeros(y.shape[0])
yProb = pd.DataFrame(np.round(y, decimals = 7), columns = ['2', '3', '4', '5'])

yHat = pd.concat([yHat, yProb], axis = 1)

yProb


#plt.hist(yArr)

#sns.heatmap(y)


,2,3,4,5
0,1.00000,0.00000,0.0,0.0
1,0.88317,0.11683,0.0,0.0
2,0.00000,1.00000,0.0,0.0
3,0.00000,1.00000,0.0,0.0
4,1.00000,0.00000,0.0,0.0
...,...,...,...,...
21421,0.00000,0.00000,1.0,0.0
21422,0.00000,0.00000,1.0,0.0
21423,0.00000,0.00000,1.0,0.0
21424,0.00000,0.00000,1.0,0.0


In [ ]:
#Testing on the other data set
import io
uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['csd.csv'])) #make sure this is named correctly or it wont work
df2.columns = ['Sequence', '1', '2', '3', '4', '5']
df2

Saving csd.csv to csd (5).csv


,Sequence,1,2,3,4,5
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,0.000000,0.085719,0.805723,0.108558,0.0
1,AAAAAAAAAVSR,0.000000,1.000000,0.000000,0.000000,0.0
2,AAAAAAALQAK,0.000000,1.000000,0.000000,0.000000,0.0
3,AAAAAWEEPSSGNGTAR,0.000000,0.872559,0.127441,0.000000,0.0
4,AAAEDVNVTFEDQQK,0.000000,1.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...
22032,YYTVFDRDNNR,0.000000,0.274602,0.725398,0.000000,0.0
22033,YYVTIIDAPGHR,0.001044,0.198856,0.800101,0.000000,0.0
22034,YYVTIIDAPGHRDFIK,0.000000,0.000000,0.000000,1.000000,0.0
22035,YYYIPQYK,0.000000,1.000000,0.000000,0.000000,0.0


In [ ]:
y2 = df2.drop('Sequence', axis=1)
sequence_col2 = df2['Sequence'].str.replace('ac-', 'ac')


aa_comp_mat2 = aaCount(sequence_col2)
mod_col2 = isMod(sequence_col2)
basic_sites2 = count_basic_sites(sequence_col2)
length_col2 = pepLength(sequence_col2)
mass_col2 = pepMass(sequence_col2)

X2 = pd.DataFrame(columns = ['Intercept', 'Mass', 'Length', 'B_Sites', 'Modified'])
X2['Intercept'] = np.ones(df2.shape[0])
X2['Mass'] = mass_col2
X2['Length'] = length_col2
X2['B_Sites'] = basic_sites2
X2['Modified'] = mod_col2

X2 = pd.concat([X2, aa_comp_mat2], axis = 1)

X2.columns

#yProb2 = model_ncg.predict_proba(X2)


#set2 = cross_val_score(model_ncg, X2, y2, scoring='accuracy', cv=cv, n_jobs=-1)

#print(f'Mean Accuracy for newton-cg model is {np.mean(set2)} and Standard Deviation is {np.std(set2)}')

Index(['Intercept', 'Mass', 'Length', 'B_Sites', 'Modified', 'Q', 'N', 'A',
       'V', 'I', 'L', 'F', 'M', 'Y', 'W', 'P', 'G', 'C', 'S', 'T', 'R', 'K',
       'H', 'E', 'D'],
      dtype='object')

In [ ]:
yTest2 = np.argmax(np.asarray(y2), axis = 1)+1
yProb2 = model_ncg.predict_proba(X2)
set2 = cross_val_score(model_ncg, X2, yTest2, scoring='accuracy', cv=cv, n_jobs=-1)
print(f'Mean Accuracy for dataset 2 with the newton-cg model is {np.mean(set2)} and Standard Deviation is {np.std(set2)}')

Mean Accuracy for dataset 2 with the newton-cg model is 0.9480419133756487 and Standard Deviation is 0.0031048050830192117


In [ ]:
yHat2 = pd.DataFrame(columns = ['1'])
yHat2['1'] = np.zeros(y2.shape[0])
yProb2 = pd.DataFrame(np.round(yProb2, decimals = 3), columns = ['2', '3', '4', '5'])

yHat2 = pd.concat([yHat2, yProb2], axis = 1)

yHat2

,1,2,3,4,5
0,0.0,0.755,0.245,0.000,0.000
1,0.0,0.998,0.002,0.000,0.000
2,0.0,0.999,0.001,0.000,0.000
3,0.0,0.933,0.067,0.000,0.000
4,0.0,0.922,0.078,0.000,0.000
...,...,...,...,...,...
22032,0.0,0.027,0.973,0.000,0.000
22033,0.0,0.017,0.983,0.001,0.000
22034,0.0,0.000,0.242,0.757,0.000
22035,0.0,1.000,0.000,0.000,0.000


In [ ]:
y2

,1,2,3,4,5
0,0.000000,0.085719,0.805723,0.108558,0.0
1,0.000000,1.000000,0.000000,0.000000,0.0
2,0.000000,1.000000,0.000000,0.000000,0.0
3,0.000000,0.872559,0.127441,0.000000,0.0
4,0.000000,1.000000,0.000000,0.000000,0.0
...,...,...,...,...,...
22032,0.000000,0.274602,0.725398,0.000000,0.0
22033,0.001044,0.198856,0.800101,0.000000,0.0
22034,0.000000,0.000000,0.000000,1.000000,0.0
22035,0.000000,1.000000,0.000000,0.000000,0.0


# Citations
Primary Resource:
https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e
https://en.wikipedia.org/wiki/Softmax_function
https://bookdown.org/egarpor/PM-UC3M/app-ext-multinomialreg.html
https://bookdown.org/chua/ber642_advanced_regression/multinomial-logistic-regression.html
https://towardsdatascience.com/predict-vs-predict-proba-scikit-learn-bdc45daa5972
https://towardsdatascience.com/multi-class-classification-one-vs-all-one-vs-one-94daed32a87b
https://www.statology.org/pandas-check-if-column-contains-string/
https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/
https://link.springer.com/article/10.1007/s10107-019-01362-7#:~:text=Another%20popular%20approach%2C%20known%20as,2%20f(x_k)%20v.
https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm